<a href="https://colab.research.google.com/github/mslisboa/ProjetoU10T3/blob/main/Implementa%C3%A7%C3%A3o_e_An%C3%A1lise_do_Algoritmo_de_K_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O dataset.zip "https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip"

--2024-11-29 04:12:45--  https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘dataset.zip’

dataset.zip             [     <=>            ]  58.18M  61.2MB/s    in 1.0s    

2024-11-29 04:12:46 (61.2 MB/s) - ‘dataset.zip’ saved [61005872]



In [2]:
!unzip dataset.zip -d dataset

Archive:  dataset.zip
 extracting: dataset/UCI HAR Dataset.names  
 extracting: dataset/UCI HAR Dataset.zip  


In [3]:
!unzip dataset/UCI\ HAR\ Dataset.zip -d "UCI HAR Dataset"

Archive:  dataset/UCI HAR Dataset.zip
   creating: UCI HAR Dataset/UCI HAR Dataset/
  inflating: UCI HAR Dataset/UCI HAR Dataset/.DS_Store  
   creating: UCI HAR Dataset/__MACOSX/
   creating: UCI HAR Dataset/__MACOSX/UCI HAR Dataset/
  inflating: UCI HAR Dataset/__MACOSX/UCI HAR Dataset/._.DS_Store  
  inflating: UCI HAR Dataset/UCI HAR Dataset/activity_labels.txt  
  inflating: UCI HAR Dataset/__MACOSX/UCI HAR Dataset/._activity_labels.txt  
  inflating: UCI HAR Dataset/UCI HAR Dataset/features.txt  
  inflating: UCI HAR Dataset/__MACOSX/UCI HAR Dataset/._features.txt  
  inflating: UCI HAR Dataset/UCI HAR Dataset/features_info.txt  
  inflating: UCI HAR Dataset/__MACOSX/UCI HAR Dataset/._features_info.txt  
  inflating: UCI HAR Dataset/UCI HAR Dataset/README.txt  
  inflating: UCI HAR Dataset/__MACOSX/UCI HAR Dataset/._README.txt  
   creating: UCI HAR Dataset/UCI HAR Dataset/test/
   creating: UCI HAR Dataset/UCI HAR Dataset/test/Inertial Signals/
  inflating: UCI HAR Dataset/UCI H

In [3]:
!pip install skimpy

In [7]:
import pandas as pd
import re

In [8]:
from skimpy import skim

In [9]:
with open("/content/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt", "r") as file:
  lines = file.readlines()

cleaned_lines = [re.sub(r'\s+', ';', line.strip()) for line in lines]

data_test = pd.read_csv("/content/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt", delimiter=";")

In [10]:
labels_test = pd.read_csv("/content/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", delimiter="\t")

In [11]:
combined_df_test = pd.concat([data_test, labels_test], axis=1)

In [12]:
skim(combined_df_test)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 2946   │ │ string      │ 1     │                                                          │
│ │ Number of columns │ 2      │ │ int64       │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓  │
│ ┃ column_name       ┃ NA   ┃ NA %    ┃ mean     ┃ sd      ┃ p0   ┃ p25   ┃ p50   ┃ p75   ┃ p100   ┃ hist     ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩  │
│ │ 5                 │    0 │       0 │    3.577 │    1.74 │    1 │     2 │     4 │     5 │      6 │  ▇▇▆▇▇▇  │  │
│ └───────────────────┴──────┴─────────┴──────────┴─────────┴──────┴───────┴───────┴───────┴────────┴──────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                           ┃ NA    ┃ NA %      ┃ words per row            ┃ total words         ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩  │
│ │   2.5717778e-001 -2.3285230e-002      │     0 │         0 │                      670 │             1977094 │  │
│ │ -1.4653762e-002 -9.3840400e-001       │       │           │                          │                     │  │
│ │ -9.2009078e-001 -6.6768331e-001       │       │           │                          │                     │  │
│ │ -9.5250112e-001 -9.2524867e-001       │       │           │                          │                     │  │
│ │ -6.7430222e-001 -8.9408755e-001       │       │           │                          │                     │  │
│ │ -5.5457721e-001 -4.6622295e-001       │       │           │                          │                     │  │
│ │ 7.1720847e-001  6.3550240e-001        │       │           │                          │                     │  │
│ │ 7.8949666e-001 -8.7776423e-001        │       │           │                          │                     │  │
│ │ -9.9776606e-001 -9.9841381e-001       │       │           │                          │                     │  │
│ │ -9.3434525e-001 -9.7566897e-001       │       │           │                          │                     │  │
│ │ -9.4982365e-001 -8.3047780e-001       │       │           │                          │                     │  │
│ │ -1.6808416e-001 -3.7899553e-001       │       │           │                          │                     │  │
│ │ 2.4621698e-001  5.2120364e-001        │       │           │                          │                     │  │
│ │ -4.8779311e-001  4.8228047e-001       │       │           │                          │                     │  │
│ │ -4.5462113e-002  2.1195505e-001       │       │           │                          │                     │  │
│ │ -1.3489443e-001  1.3085848e-001       │       │           │                          │                     │  │
│ │ -1.4176313e-002 -1.0597085e-001       │       │     

In [13]:
with open("/content/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt", "r") as file:
  lines = file.readlines()

cleaned_lines = [re.sub(r'\s+', ';', line.strip()) for line in lines]

data_train = pd.read_csv("/content/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt", delimiter=";")

In [14]:
labels_train = pd.read_csv("/content/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt", delimiter="\t")

In [15]:
combined_df_train = pd.concat([data_train, labels_train], axis=1)

In [17]:
skim(combined_df_train)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 2946   │ │ string      │ 1     │                                                          │
│ │ Number of columns │ 2      │ │ int64       │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓  │
│ ┃ column_name       ┃ NA   ┃ NA %    ┃ mean     ┃ sd      ┃ p0   ┃ p25   ┃ p50   ┃ p75   ┃ p100   ┃ hist     ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩  │
│ │ 5                 │    0 │       0 │    3.577 │    1.74 │    1 │     2 │     4 │     5 │      6 │  ▇▇▆▇▇▇  │  │
│ └───────────────────┴──────┴─────────┴──────────┴─────────┴──────┴───────┴───────┴───────┴────────┴──────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                           ┃ NA    ┃ NA %      ┃ words per row            ┃ total words         ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩  │
│ │   2.5717778e-001 -2.3285230e-002      │     0 │         0 │                      670 │             1977094 │  │
│ │ -1.4653762e-002 -9.3840400e-001       │       │           │                          │                     │  │
│ │ -9.2009078e-001 -6.6768331e-001       │       │           │                          │                     │  │
│ │ -9.5250112e-001 -9.2524867e-001       │       │           │                          │                     │  │
│ │ -6.7430222e-001 -8.9408755e-001       │       │           │                          │                     │  │
│ │ -5.5457721e-001 -4.6622295e-001       │       │           │                          │                     │  │
│ │ 7.1720847e-001  6.3550240e-001        │       │           │                          │                     │  │
│ │ 7.8949666e-001 -8.7776423e-001        │       │           │                          │                     │  │
│ │ -9.9776606e-001 -9.9841381e-001       │       │           │                          │                     │  │
│ │ -9.3434525e-001 -9.7566897e-001       │       │           │                          │                     │  │
│ │ -9.4982365e-001 -8.3047780e-001       │       │           │                          │                     │  │
│ │ -1.6808416e-001 -3.7899553e-001       │       │           │                          │                     │  │
│ │ 2.4621698e-001  5.2120364e-001        │       │           │                          │                     │  │
│ │ -4.8779311e-001  4.8228047e-001       │       │           │                          │                     │  │
│ │ -4.5462113e-002  2.1195505e-001       │       │           │                          │                     │  │
│ │ -1.3489443e-001  1.3085848e-001       │       │           │                          │                     │  │
│ │ -1.4176313e-002 -1.0597085e-001       │       │     